# Testing Agentic approach to rubric evaluation

In [1]:
import openai
import glob, os, time, json, re, http, asyncio
from pydantic import BaseModel, Field
from enum import Enum
from agents import Agent, Runner, Tool, trace
from pydantic import BaseModel, Field
from enum import Enum
from typing import List

# Escape Codes para imprimir mensajes con color
Rd = "\033[1;31m"
Grn = "\033[1;32m"
Ylw = "\033[1;33m"
Blu = "\033[1;34m"
Mag = "\033[1;35m"
Rst = "\033[0m"

spacer = f"{80*'-'}"

Rubric Evaluation Agent

In [71]:
class EvidenceStrength(str, Enum):
    EXPLICIT_STRONG = "Explicit & Strong Evidence"
    EXPLICIT_WEAK = "Explicit but Weak Evidence"
    IMPLICIT = "Implicit Evidence or Assumption"
    NONE = "No Evidence Provided"
    NOT_REQUIRED = "No Evidence Required (Trivial or Common Knowledge)"

class Evidence(BaseModel):
    """Formato de evidencia, que indica si es claro y sustentado con evidencia."""
    evidence: str = Field(description="Extracto exacto del texto que justifica el criterio evaluado. Debe ser un fragmento tal cual como viene en el texto provisto.")
    clarity: str = Field(description="Indica si la evidencia es clara y coherente o si le falta claridad. Ej: 'Claridad adecuada' o 'Falta de explicación clara'.")
    referencia: EvidenceStrength = Field(description=(
            "Referencia o sustento científico de la evidencia. Puede ser cita bibliográfica, inferencia, "
            "conocimiento asumido, evidencia débil, o trivial. Las opciones posibles son:\n"
            "1. 'Explicit & Strong Evidence': Evidencia clara y robusta, con fuente confiable.\n"
            "2. 'Explicit but Weak Evidence': Fuente directa, pero débil o ambigua.\n"
            "3. 'Implicit Evidence or Assumption': Basado en conocimiento tácito o asumido, sin cita directa.\n"
            "4. 'No Evidence Provided': No se presenta evidencia respaldatoria.\n"
            "5. 'No Evidence Required (Trivial or Common Knowledge)': Conocimiento común, no requiere evidencia."
        ))
    manifestation: str = Field(description=(
    "Forma en que se manifiesta la evidencia dentro del documento evaluado. "
    "Las formas más comunes son:\n"
    "- 'Afirmación textual directa': Una afirmación explícita encontrada en el texto.\n"
    "- 'Hipervínculo': Enlace incluido en el texto que dirige a evidencia externa.\n"
    "- 'Cita bibliográfica': Referencia académica o técnica incluida como fuente.\n"
    "- 'Documento adjunto o anexo': Mención de un documento externo relacionado.\n"
    "- 'Otro': En caso de que la evidencia adopte una forma no convencional (especificar)."
))

class TechnicalMeritRubicCriterion(BaseModel):
    """Rúbrica de Criterios para evaluar el mérito técnico del proyecto."""
    
    technical_evidence: List[Evidence] = Field(
        ...,
        description=(
            "Evidencias del criterio relacionadas con la base técnica del proyecto. "
            "Debe incluir pruebas de la solidez técnica, enfoque metodológico, y coherencia de los fundamentos "
            "que sustentan el proyecto. Ejemplos incluyen modelos técnicos, análisis de viabilidad, "
            "y detalles sobre las tecnologías empleadas."
        )
    )
    scientific_justification: List[Evidence] = Field(
        ...,
        description=(
            "Evidencias relacionadas con la justificación científica que respalda el proyecto. "
            "Debe incluir la validación científica, teorías aplicadas, estudios previos relevantes, y el uso "
            "de referencias académicas que apoyan la propuesta. Ejemplos incluyen citas bibliográficas, "
            "resultados de investigaciones previas, o documentación que valida la base científica del proyecto."
        )
    )

class TechnicalSolutionRubicCriterion(BaseModel):
    """Rúbrica de Criterios para evaluar la claridad y coherencia en la solución técnica propuesta."""
    
    solution_clarity: List[Evidence] = Field(
        ...,
        description="Evidencias relacionadas con la claridad de la solución propuesta para abordar el problema."
    )
    solution_coherence: List[Evidence] = Field(
        ...,
        description="Evidencias relacionadas con la coherencia de la solución y su alineación con el problema."
    )
    justification: List[Evidence] = Field(
        ...,
        description="Evidencias que sustentan la justificación de la solución técnica propuesta."
    )

class TechnologyMaturityRubicCriterion(BaseModel):
    """Rúbrica de Criterios para evaluar el nivel de maduración tecnológica según la escala TRL."""
    
    technology_maturity_level: List[Evidence] = Field(
        ...,
        description="Evidencias que describen el nivel de maduración tecnológica del proyecto según la escala TRL."
    )
    
    evidence_of_progress: List[Evidence] = Field(
        ...,
        description="Evidencias de los avances hacia el cumplimiento de TRL 4 y de los pasos realizados hacia un TRL superior."
    )
    
    validation_evidence: List[Evidence] = Field(
        ...,
        description="Evidencias sobre la validación realizada a la tecnología en las distintas etapas de desarrollo."
    )
    
    plan_for_advancement: List[Evidence] = Field(
        ...,
        description="Evidencias de los planes estructurados para avanzar a TRL 5 o superior, con pasos claros y definidos."
    )

class ImplementationFeasibilityRubicCriterion(BaseModel):
    """Rúbrica de Criterios para evaluar la viabilidad de la implementación considerando recursos y contexto."""
    
    implementation_feasibility: List[Evidence] = Field(
        ...,
        description="Evidencias que describen la viabilidad de la implementación en función de los recursos disponibles y el contexto."
    )
    
    resource_availability: List[Evidence] = Field(
        ...,
        description="Evidencias sobre la disponibilidad de recursos necesarios para la implementación del proyecto."
    )
    
    risk_assessment: List[Evidence] = Field(
        ...,
        description="Evidencias que describen los riesgos asociados con la implementación y las estrategias para mitigar estos riesgos."
    )
    
    execution_strategy: List[Evidence] = Field(
        ...,
        description="Evidencias que describen la estrategia clara de ejecución del proyecto, con pasos bien definidos y cronograma adecuado."
    )
    
    uncertainty_management: List[Evidence] = Field(
        ...,
        description="Evidencias que muestran cómo se gestionan las incertidumbres y áreas de mejora dentro del proyecto."
    )

class TeamCompositionRubicCriterion(BaseModel):
    """Rúbrica de Criterios para evaluar la experiencia y trayectoria del equipo proponente en proyectos similares."""
    
    team_experience: List[Evidence] = Field(
        ...,
        description="Evidencias que describen la experiencia del equipo en proyectos similares y su capacidad técnica."
    )
    
    relevant_background: List[Evidence] = Field(
        ...,
        description="Evidencias sobre el historial y antecedentes relevantes de los miembros del equipo en áreas clave."
    )
    
    team_complementarity: List[Evidence] = Field(
        ...,
        description="Evidencias que muestran la complementariedad de las habilidades dentro del equipo, asegurando una cobertura adecuada de todas las áreas necesarias para el proyecto."
    )
    
    leadership_capabilities: List[Evidence] = Field(
        ...,
        description="Evidencias que demuestran las capacidades de liderazgo del equipo, incluyendo la gestión y la toma de decisiones."
    )
    
    individual_expertise: List[Evidence] = Field(
        ...,
        description="Evidencias que destacan la experiencia individual de los miembros clave en el equipo, especialmente en áreas críticas para el proyecto."
    )

class BudgetJustificationRubicCriterion(BaseModel):
    """Rúbrica de Criterios para evaluar la distribución y justificación del presupuesto asignado al proyecto."""
    
    budget_clarity: List[Evidence] = Field(
        ...,
        description="Evidencias que demuestran que la distribución del presupuesto está claramente definida y alineada con los objetivos del proyecto."
    )
    
    resource_distribution: List[Evidence] = Field(
        ...,
        description="Evidencias sobre la distribución de los recursos del presupuesto de forma coherente y equilibrada entre las diferentes necesidades del proyecto."
    )
    
    justification_of_expenses: List[Evidence] = Field(
        ...,
        description="Evidencias que muestran cómo se justifica cada gasto dentro del presupuesto, asegurando que cada partida tiene un propósito claro y relevante para el proyecto."
    )
    
    optimization_opportunities: List[Evidence] = Field(
        ...,
        description="Evidencias que evalúan si el presupuesto ha sido optimizado adecuadamente o si existen áreas donde se podrían reducir gastos sin afectar el proyecto."
    )
    
    budget_alignment: List[Evidence] = Field(
        ...,
        description="Evidencias que confirman que el presupuesto está alineado con los objetivos y la estrategia del proyecto, asegurando que los recursos se destinan donde son más necesarios."
    )
    
    completeness: List[Evidence] = Field(
        ...,
        description="Evidencias que demuestran que el presupuesto está completamente detallado, sin omisiones ni partidas faltantes."
    )

class ExpectedImpactsMonitoringRubicCriterion(BaseModel):
    """Rúbrica de Criterios para evaluar la definición de impactos esperados y su metodología de monitoreo."""
    
    impact_definition: List[Evidence] = Field(
        ...,
        description="Evidencias que demuestran que los impactos esperados del proyecto están claramente definidos, detallados y alineados con los objetivos del proyecto."
    )
    
    impact_relevance: List[Evidence] = Field(
        ...,
        description="Evidencias que muestran que los impactos esperados están directamente relacionados con el problema a resolver, y cómo la solución propuesta contribuirá de manera significativa."
    )
    
    monitoring_methodology: List[Evidence] = Field(
        ...,
        description="Evidencias sobre la metodología de monitoreo propuesta para seguir el progreso de los impactos. Esto incluye métodos de seguimiento, indicadores y herramientas que se usarán."
    )
    
    monitoring_plan: List[Evidence] = Field(
        ...,
        description="Evidencias de que el plan de monitoreo es detallado y cubre las etapas necesarias del proyecto, incluyendo tiempos y formas de medición de los impactos."
    )
    
    impact_assessment: List[Evidence] = Field(
        ...,
        description="Evidencias que indican cómo se evaluarán los impactos esperados, incluyendo cómo se medirá su éxito y qué métricas se utilizarán para su evaluación."
    )
    
    adjustment_mechanism: List[Evidence] = Field(
        ...,
        description="Evidencias que describen los mecanismos de ajuste en caso de que los impactos no se logren según lo planeado, incluyendo acciones correctivas o alternativas."
    )

class WorkPlanRubicCriterion(BaseModel):
    """Rúbrica de Criterios para evaluar el plan de trabajo propuesto con hitos y cronograma."""
    
    work_plan_structure: List[Evidence] = Field(
        ...,
        description="Evidencias que demuestran que el plan de trabajo está bien estructurado, con una clara descripción de las tareas, los entregables y las fases del proyecto."
    )
    
    milestones_definition: List[Evidence] = Field(
        ...,
        description="Evidencias que muestran que los hitos del proyecto están claramente definidos, con objetivos alcanzables y medibles para cada fase."
    )
    
    timeline_and_schedule: List[Evidence] = Field(
        ...,
        description="Evidencias que detallan el cronograma del proyecto, indicando los tiempos estimados para cada fase o tarea, y si son realistas y alcanzables."
    )
    
    task_distribution: List[Evidence] = Field(
        ...,
        description="Evidencias que demuestran cómo se distribuyen las tareas entre los miembros del equipo y cómo se planifica el uso de recursos en función de los hitos."
    )
    
    risk_and_contingency_planning: List[Evidence] = Field(
        ...,
        description="Evidencias sobre la identificación de posibles riesgos y la existencia de un plan de contingencia para mitigarlos y asegurar el cumplimiento de los hitos."
    )
    
    deliverable_quality: List[Evidence] = Field(
        ...,
        description="Evidencias que confirman que los entregables del proyecto están bien definidos, tienen criterios claros de calidad y son evaluables."
    )

class IntellectualPropertyRubicCriterion(BaseModel):
    """Rúbrica de Criterios para evaluar la propiedad intelectual en proceso o concedida."""
    
    ip_generation_potential: List[Evidence] = Field(
        ...,
        description="Evidencias que demuestran el potencial de generación de propiedad intelectual en el proyecto, como innovaciones tecnológicas, procesos, o productos originales."
    )
    
    ip_protection_evidence: List[Evidence] = Field(
        ...,
        description="Evidencias de que se ha realizado algún avance en la protección de la propiedad intelectual, como solicitudes de patentes, marcas, derechos de autor, u otros mecanismos de protección."
    )
    
    ip_trademark_or_patent_status: List[Evidence] = Field(
        ...,
        description="Evidencias del estado actual de las solicitudes de protección intelectual, como patentes o marcas registradas, y si están en trámite o si ya han sido concedidas."
    )
    
    ip_management_strategy: List[Evidence] = Field(
        ...,
        description="Evidencias sobre la existencia de una estrategia clara para la gestión y protección de la propiedad intelectual, como planificación, análisis de mercado, o acuerdos de licencia."
    )
    
    ip_granted_or_advanced_status: List[Evidence] = Field(
        ...,
        description="Evidencias que indican que la propiedad intelectual ha sido concedida o está en proceso avanzado, mostrando la protección definida para los derechos relacionados."
    )


class MaturityRubric(BaseModel):
    """Formato de Rúbrica, para juntar las evidencias relacionadas a la madurez de un proyecto científico."""
    # Criterio de mérito técnico
    merito_tecnico: TechnicalMeritRubicCriterion = Field(
        description="Evidencias relacionadas a la base técnica del proyecto y su justificación científica."
    )
    
    # Criterio de solución técnica
    solucion_tecnica: TechnicalSolutionRubicCriterion = Field(
        description="Evidencias relacionadas a la claridad y coherencia de la solución técnica propuesta para abordar el problema."
    )
    
    # Criterio de madurez tecnológica
    madurez_tecnologica: TechnologyMaturityRubicCriterion = Field(
        description="Evidencias relacionadas al nivel de maduración tecnológica del proyecto según la escala TRL."
    )
    
    # Criterio de viabilidad de implementación
    viabilidad_implementacion: ImplementationFeasibilityRubicCriterion = Field(
        description="Evidencias relacionadas a la viabilidad de implementación considerando recursos y contexto."
    )
    
    # Criterio de composición del equipo
    equipo_proponente: TeamCompositionRubicCriterion = Field(
        description="Evidencias relacionadas a la experiencia y trayectoria del equipo en proyectos similares."
    )
    
    # Criterio de justificación del presupuesto
    justificacion_presupuesto: BudgetJustificationRubicCriterion = Field(
        description="Evidencias relacionadas a la distribución y justificación del presupuesto asignado al proyecto."
    )
    
    # Criterio de impactos esperados y su metodología de monitoreo
    impactos_monitoreo: ExpectedImpactsMonitoringRubicCriterion = Field(
        description="Evidencias relacionadas a la definición clara de los impactos esperados y su monitoreo."
    )
    
    # Criterio de plan de trabajo propuesto con hitos y calendario
    plan_trabajo: WorkPlanRubicCriterion = Field(
        description="Evidencias relacionadas a la claridad del plan de trabajo con entregables y cronograma."
    )
    
    # Criterio de propiedad intelectual en proceso o concedida
    propiedad_intelectual: IntellectualPropertyRubicCriterion = Field(
        description="Evidencias relacionadas a la existencia o potencial de generación de propiedad intelectual."
    )

In [72]:
rubric_evaluator_agent = Agent(
    name="Rubric Evaluator Agent",
    instructions="""
    Description:
    The Rubric Evaluator Agent conducts a rigorous initial evaluation of a project proposal based on a structured maturity assessment rubric. Its core function is to extract, assess, and organize **hard evidence** from the proposal, aligning it to each rubric criterion. This agent prioritizes **thoroughness and honesty**: it does **not assume or infer evidence** and is **not obligated to complete all rubric fields** if no solid evidence exists.

    Approach:
    - **Extensively search for explicit, traceable evidence** related to each criterion in the maturity rubric.
    - **Avoid speculative judgments**: if there is no clear or strong evidence, the agent must **leave the field unscored or flag it accordingly**.
    - If evidence is strong, the agent should **confidently acknowledge it**. If evidence is weak, limited, or ambiguous, this should also be made clear.

    Inputs:
    1. A project or proposal document containing technical, methodological, organizational, and contextual information.
    2. A predefined maturity rubric with clearly described criteria (e.g., TRL, technical merit, feasibility, impacts, team experience).

    Output:
    A structured object (`MaturityRubric`) where each field corresponds to a rubric criterion and contains an evidence-grounded analysis. The output:
    - Highlights **whether the criterion is strongly supported, weakly supported, or unsupported**.
    - May leave fields empty if **no verifiable evidence is found**.

    Guidelines:
    - The agent **does not evaluate based on potential**—only on present, explicit information.
    - The output should include direct references to the text or reasoning showing **why** a criterion is (or isn't) fulfilled.
    - Do **not be afraid to say the evidence is insufficient**, vague, or absent.
    
    Workflow Position:
    This agent works **prior to** the deliberation phase with other agents (e.g., Proponent, Devil’s Advocate). It establishes a **fact-based baseline**, grounded in evidence, that serves to inform and constrain further discussion.

    Adjustment:
    The agent can revise its output if new documentation is submitted or clarification is provided later in the evaluation process.
    """,
    tools=[],
    output_type=MaturityRubric,
    model="gpt-4o-mini-2024-07-18",
)



In [73]:
with open('./QRO/779_Aguadelluvia.txt', 'rb') as file:
    file_content = file.read()
    file_content = file_content.decode('utf-8', errors='ignore')

In [74]:
with trace("Prueba de Extracción de Evidencias"):
    result = await Runner.run(
        rubric_evaluator_agent,
        input=file_content,
        )


In [75]:
for rubric, evidences in result.final_output.model_dump().items():
    print(f"➤ {Ylw}{rubric}{Rst}")
    for evidence_type, evidence_list in evidences.items():
        print(f"\t➕{Grn}{evidence_type}{Rst}:")
        for i, evidence in enumerate(evidence_list, start=1):
            print(f"\t  {i}. ✅{evidence['evidence']}")
            print(f"\t     {Mag}Evidence Manifestation:{Rst} {evidence['manifestation']}")
            print(f"\t     {Mag}Clarity:{Rst} {evidence['clarity']} {Mag}Fortaleza de Evidencia:{Rst} {evidence['referencia'].value}\n")
    print(spacer)

➤ merito_tecnico
	➕technical_evidence:
	  1. ✅La propuesta integra un sistema de predicción de precipitaciones basado en modelos como GFS y WRF, con el desarrollo de reactores eficientes de oxidación electroquímica.
	     Evidence Manifestation: Afirmación textual directa
	     Clarity: Claridad adecuada Fortaleza de Evidencia: Explicit & Strong Evidence

	  2. ✅Se han desarrollado reactores electroquímicos que han sido validados en laboratorio y en condiciones reales, alcanzando un nivel TRL4.
	     Evidence Manifestation: Afirmación textual directa
	     Clarity: Claridad adecuada Fortaleza de Evidencia: Explicit & Strong Evidence

	➕scientific_justification:
	  1. ✅El proyecto menciona estudios previos y patentes que respaldan la viabilidad de la tecnología de oxidación electroquímica para el tratamiento de aguas.
	     Evidence Manifestation: Cita bibliográfica
	     Clarity: Claridad adecuada Fortaleza de Evidencia: Explicit & Strong Evidence

-------------------------------------

Multi Agent Argumentation Framework

In [ ]:
from agents import function_tool

@function_tool
def original_document() -> str:
    """Provides the original document, that desires funding from our institution."""
    return file_content

@function_tool
def rubric_facts() -> MaturityRubric:
    """Provides the list of found facts that align to the rubric. Contains evidence extracted from the original document. Done previously by an evaluator."""
    return result.final_output

@function_tool
def rubric_format() -> str:
    """Thie is the rubric for evluation."""
    text = """
    # Rúbrica de Evaluación de Proyectos de Investigación

    ## 1. Mérito técnico
    _Evaluación de la base científica y técnica del proyecto._

    - **5 puntos**: Justificación técnica excelente, con respaldo sólido y claridad.
    - **4 puntos**: Base técnica bien sustentada.
    - **3 puntos**: Justificación aceptable, aunque con áreas de mejora.
    - **2 puntos**: Fundamento débil o poco claro.
    - **1 punto**: Justificación técnica vaga o sin referencias claras.
    - **0 puntos**: Sin justificación técnica.

    ---

    ## 2. Solución técnica propuesta
    _Claridad, coherencia e innovación de la solución planteada al problema._

    - **5 puntos**: Solución innovadora, bien formulada y alineada al problema.
    - **4 puntos**: Solución clara y estructurada, con justificación sólida.
    - **3 puntos**: Solución adecuada pero con áreas poco desarrolladas.
    - **2 puntos**: Solución identificable, pero con poca claridad o detalle.
    - **1 punto**: Planteamiento vago o no relacionado con el problema.
    - **0 puntos**: Sin una solución clara o coherente.

    ---

    ## 3. Nivel de madurez tecnológica (TRL ≥ 4 preferente)
    _Estado de desarrollo tecnológico del proyecto._

    - **5 puntos**: TRL 4 con evidencia sólida y plan claro hacia TRL 5 o más.
    - **4 puntos**: TRL 4 validado en laboratorio.
    - **3 puntos**: TRL 4 con validación parcial.
    - **2 puntos**: TRL 3 con intención de avanzar a TRL 4, validación limitada.
    - **1 punto**: TRL 3 sin evidencia de avance.
    - **0 puntos**: TRL menor a 3 o sin evidencia.

    ---

    ## 4. Viabilidad de implementación
    _Factibilidad técnica y operativa con los recursos disponibles._

    - **5 puntos**: Proyecto claramente viable, con estrategia de ejecución.
    - **4 puntos**: Implementación bien fundamentada, con riesgos controlables.
    - **3 puntos**: Viabilidad aceptable, pero con incertidumbres.
    - **2 puntos**: Posible, pero con riesgos altos sin mitigación clara.
    - **1 punto**: Dificultades significativas para implementarse.
    - **0 puntos**: No viable en las condiciones actuales.

    ---

    ## 5. Equipo proponente (perfil y trayectoria)
    _Experiencia del equipo en proyectos similares._

    - **5 puntos**: Equipo altamente calificado, con trayectoria sólida y liderazgo.
    - **4 puntos**: Equipo con experiencia y capacidades complementarias.
    - **3 puntos**: Equipo adecuado, con áreas de mejora en trayectoria.
    - **2 puntos**: Algún miembro con experiencia, pero equipo débil.
    - **1 punto**: Experiencia limitada o poco clara.
    - **0 puntos**: Sin experiencia ni evidencia de capacidad técnica.

    ---

    ## 6. Justificación del presupuesto
    _Coherencia entre los recursos solicitados y los objetivos._

    - **5 puntos**: Presupuesto detallado, óptimo y bien justificado.
    - **4 puntos**: Presupuesto estructurado y alineado a los objetivos.
    - **3 puntos**: Presupuesto aceptable, con oportunidades de optimización.
    - **2 puntos**: Justificación parcial, con áreas no justificadas.
    - **1 punto**: Distribución poco clara o inconsistente.
    - **0 puntos**: Presupuesto sin justificación o con errores graves.

    ---

    ## 7. Impactos esperados y monitoreo
    _Definición clara de impactos y su seguimiento._

    - **5 puntos**: Impactos sólidos con monitoreo definido y estructurado.
    - **4 puntos**: Impactos bien definidos con monitoreo adecuado.
    - **3 puntos**: Impactos razonables, con metodología mejorable.
    - **2 puntos**: Impactos definidos, pero monitoreo débil.
    - **1 punto**: Impactos mencionados sin relación clara con el problema.
    - **0 puntos**: No se definen impactos ni forma de medirlos.

    ---

    ## 8. Plan de trabajo (hitos y calendario)
    _Claridad del cronograma y entregables._

    - **5 puntos**: Plan detallado, con hitos definidos y calendario realista.
    - **4 puntos**: Plan estructurado, con hitos claros y alcanzables.
    - **3 puntos**: Hitos definidos, pero calendario poco claro.
    - **2 puntos**: Plan poco estructurado o incompleto.
    - **1 punto**: Plan desorganizado o sin entregables definidos.
    - **0 puntos**: No presenta plan de trabajo.

    ---

    ## 9. Propiedad intelectual (preferente)
    _Potencial o existencia de generación de propiedad intelectual._

    - **5 puntos**: Propiedad intelectual concedida o proceso avanzado y claro.
    - **4 puntos**: En trámite, con estrategia clara.
    - **3 puntos**: Avances hacia protección identificados.
    - **2 puntos**: Posible generación, pero sin evidencia clara.
    - **1 punto**: Bajo potencial de generación.
    - **0 puntos**: No se considera o no hay intención.

    """
    return text

In [87]:
proponent_agent = Agent(
    name="Proponent Agent",
    instructions="""
    instructions:
    The Proponent Agent presents the proposal or document for review, providing the strongest rationale and evidence in favor of the proposed project or concept. This agent is tasked with making a compelling case for why the proposal should be accepted.

    Inputs:
    1. The document containing the proposal, evidence, and background information.

    2. Any supporting data, previous success stories, or detailed methodologies used in the proposal.

    Objective:
    1. Present the proposal in a clear and persuasive manner, highlighting key strengths and justifications for its implementation.
    2. Respond to critiques from other agents during the review process.

    Adjust to Feedback:
    1. Incorporate feedback from the critique rounds to strengthen areas of the proposal that were found to be lacking.
    2. Address weaknesses or gaps in evidence, clarifying and providing additional supporting materials where necessary.
    
    """,
    tools=[original_document, rubric],
    model="gpt-4o-2024-08-06"
)

devils_advocate_agent = Agent(
    name="Devil's Advocate Agent",
    instructions="""
    instructions:
    The Devil’s Advocate Agent identifies potential weak points, risks, and inconsistencies in the proposal. Their role is to challenge the assumptions and conclusions made in the document, uncovering areas that might lead to failure or unintended negative outcomes.

    Inputs:
    1. The proposal document.
    2. Rubric highlighting areas of concern or weaknesses in the evidence.

    Objective:
    1. Find flaws or inconsistencies within the proposal, pointing out where the reasoning is not robust or where assumptions may be unwarranted.
    2. Introduce alternative viewpoints or counterarguments to test the proposal’s validity.

    Adjust to Feedback:
    1. Reflect on the responses to their critiques during the Cross-Critique phase. If their challenges are found to be weak or unjustified, adjust arguments or refine the critique to make it more relevant.
    2. Reassess the proposal and its defense after incorporating the Proponent's responses.
    """,
    tools=[original_document, rubric],
    model="gpt-4o-2024-08-06"
)

evidence_reviewer_agent = Agent(
    name="Evidence Reviewer Agent",
    instructions=""" 
    instructions:
    The Evidence Reviewer Agent critically examines the evidence provided in support of the proposal. This agent focuses on verifying the quality, validity, and relevance of the data and assumptions used to justify the proposal.

    Inputs:
    1. The proposal document with the presented evidence.
    2. Rubric identifying evidence types required (e.g., statistical analysis, expert opinion, case studies, etc.).

    Objective:
    1. Challenge the assumptions by reviewing the presented evidence and identifying gaps or areas where evidence is insufficient, misleading, or absent.
    2. Suggest additional evidence or data sources that may strengthen the proposal.

    Adjust to Feedback:
    1. If the Proponent can provide more credible or relevant evidence, adjust the critique to reflect new findings.
    2. Revise the evaluation criteria based on the Proponent's ability to counter the evidence critique.
    """,
    tools=[original_document, rubric],
    model="gpt-4o-2024-08-06"
)

feasibility_agent = Agent(
    name="Feasibility Agent",
    instructions=""" 
    instructions:
    The Feasibility Agent evaluates whether the proposal is practically executable within the given context. This includes assessing the resources, time, technology, and expertise needed to implement the proposal and whether the proposal is viable in real-world conditions.

    Inputs:
    1. The proposal document and context information (budget, timeline, resource constraints).
    2. Rubric identifying feasibility requirements (e.g., required technology, skillsets, timeframe, etc.).

    Objective:
    1. Assess the proposal’s practicality and implementation challenges, such as cost, resource constraints, timeline, or technological feasibility.
    2. Identify potential barriers to successful implementation.

    Adjust to Feedback:
    1. Reassess the proposal based on the Proponent's explanations of resource allocation or potential adjustments to the project.
    2. Refine feasibility concerns if additional supporting information is provided, such as funding or resource guarantees.
    """,
    tools=[original_document, rubric],
    model="gpt-4o-2024-08-06"
)

ethics_impact_agent = Agent(
    name="Ethics/Impact Agent",
    instructions=""" 
    instructions:
    The Ethics/Impact Agent evaluates the broader ethical implications and potential societal impacts of the proposal. This agent considers issues like environmental impact, fairness, equity, privacy, safety, and long-term consequences.

    Inputs:
    1. The proposal document, with particular attention to the ethical considerations and potential risks outlined.
    2. Rubric highlighting required ethical and societal considerations.

    Objective:
    1. Identify any ethical dilemmas, unintended societal consequences, or risks associated with the proposal.
    2. Challenge the proposal on moral grounds, proposing alternative approaches where appropriate.

    Adjust to Feedback:
    1. If the Proponent addresses concerns with mitigation strategies or revised ethical assessments, adjust critiques accordingly.
    2. Reevaluate the ethical considerations based on new information or proposed changes to the project.
    """,
    tools=[original_document, rubric],
    model="gpt-4o-2024-08-06"
)

synthesis_judge_agent = Agent(
    name="Synthesis Judge Agent",
    instructions=""" 
    instructions:
    The Synthesis Judge Agent is responsible for synthesizing all the arguments, critiques, and defenses and ultimately making a decision regarding the proposal. This agent evaluates all the inputs and provides a final score or decision (pass, fail, defer) based on the quality of the proposal, the strength of the critiques, and the Proponent’s responses.

    Inputs:
    1. All previous discussions, critiques, and defenses.
    2. The proposal document, including all adjustments made during the critique rounds.

    Objective:
    1. Weigh all arguments fairly, taking into account the strength of evidence, feasibility, ethics, and risks.
    2. Provide a final decision based on the quality of the proposal and the validity of the critiques and defenses.

    Adjust to Feedback:
    1. If new arguments or adjustments are introduced by the Proponent, the Synthesis Judge must incorporate them into their final evaluation.
    2. Continually update the decision based on new insights, especially during the Cross-Critique and Defense rounds.
    """,
    tools=[original_document, rubric],
    model="gpt-4o-2024-08-06"
    
)


In [88]:
with trace("MAAF Evaluation"):
    result = await Runner.run(
        proponent_agent,
        input="¿What is your assesement?",
        )


In [90]:
print(result.final_output)

Based on the provided proposal document and rubric, here’s a comprehensive assessment:

### Key Strengths of the Proposal

1. **Innovative Technology**:
   - The proposal introduces advanced electrochemical oxidation technology aimed at rainwater harvesting and greywater treatment. This innovative approach incorporates electro-Fenton processes and solar energy integration, making it environmentally friendly and sustainable.

2. **Economy Circular Approach**:
   - The technology is designed to maximize water use efficiency, contributing to a circular economy. It involves using harvested rainwater and treated greywater, reducing reliance on potable water, which is especially beneficial in water-scarce regions.

3. **Collaborative and Multidisciplinary Effort**:
   - The collaboration between multiple prestigious institutions such as UNAM, UAQ, and CIDETEQ brings together diverse expertise from meteorology, chemistry, and engineering to ensure the project’s success.

4. **Real-World Appli